In [3]:
import sys
import pickle
import pprint
pp = pprint.PrettyPrinter(indent=0)

import numpy as np
from functools import reduce

# Local Modules

### utilities.py

In [14]:
# utilties.py

def get_attributes(pobj):
    """Given any python object returns its data attributes or methods.
    When the input is an instance it returns instance attributes.
    When the input is the Class it returns extended/added methods.

    Args:
        pobj (:obj:`Maker`): A python object.

    Returns:
        (:obj:`list` of :obj:`str`): list of attribute names.

    Raises:
        TypeError: Raised if an object type without a __dict__ attribute is given.
        
    """

    if '__dict__' not in dir(pobj):
        raise TypeError('A Python object with attributes is expected.')

    attrs = {k:v for k,v in pobj.__dict__.items() if not k.startswith('_')}
    return [k for k, v in attrs.items() if type(v) != 'function']


def get_random_item(adict):
    """The function returns a random item from a dictionary.

    Args:
        pobj (:obj:`dict`): A python dictionary.

    Returns:
         None: If the the input is an empty dictionary.
        (:obj:`dict`): A random dictionary item in the form {k:v}.
        
     Raises:
        TypeError: Raised if a non `dict` is given.
    
    """
    if not isinstance(adict, dict):
        raise TypeError('A Python dictionary object is expected.') 
    N = len(adict)
    if N == 0: return
    ind = int(np.random.rand() * (N - 1))
    k,v = list(adict.items())[ind]
    return {k:v}

def get_sample(adict, n):
    """The function returns a number of random items from a dictionary.

    Args:
        pobj (:obj:`dict`): A python dictionary.
        n (:obj:`int`): A number of items.

    Returns:
         None: If the the input is an empty dictionary.
        (:obj:`dict`): A dictionary with item in the form {k:v}.
        
     Raises:
        TypeError: Raised if a non `dict` is given.
    
    """
    if not isinstance(adict, dict):
        raise TypeError('A Python dictionary object is expected.') 
    N = len(adict)
    if N == 0: return
    if n > N: n = N
    sample = dict()
    while n > 0:
        ind = int(np.random.rand() * (N - 1))
        k,v = list(adict.items())[ind]
        sample[k] = v
        n -= 1
    return sample

def form_network_relations(connections):
    """Given a list of attribute/connection sets, it returns a co-occurance matrix of the attributes
        observed in the sets.
    
    Note: It should be noted that the diagonal, which would be occurance count is empty and this frequency
        info can be accessed by the 'f' key of the matrix/dictionary.
  
    Args:
        pobj (:obj:`list` of :obj:`set` of :obj:`str`): Co-oocurance data.

    Returns:
        (:obj:`dict`): A dictionary which contains a network data where the keys are the nodes (attributes)
        and values are a dictionary which contains the degree and the weighted links of the node.
    """
    attrs = connections
    alltags = set()
    for k in attrs:
        alltags = alltags.union(set(k))
        
    occurance = dict()
    for t in alltags:
        occurance[t] = {'f':0,'co':{}}
        
    for kset in attrs:
        for k in kset:
            occurance[k]['f'] += 1
            for t in kset:
                if t == k: continue
                if t in occurance[k]['co'].keys():
                    occurance[k]['co'][t] += 1
                else:
                    occurance[k]['co'][t] = 1            
    return occurance


def load_skill_cooccurance(pobj = './data/skill_occurance.pickle'):
    """The function loads the skill occurances.
    
    Note:
        The occurances can be driven either from projects or profiles.
        However, their use implications are different.

    Args:
        pobj (:obj:`string`): The string pointer to the (:obj:`pickle`) python dictionary.
            The dictionary is curated from surveys or other external sources.

    Returns:
        (:obj:`list` of :obj:`set`): The list of skills sets.
    
    """
    fname = pobj
    with open(fname, 'rb') as f:
        projects = pickle.load(f)
    return projects


def load_member_profiles(pobj = './data/crm_profile.pickle'):
    """The utility loads member profile data.

    Args:
        pobj (:obj:`str`): The string pointer to the serialized python dictionary (:obj:`pickle`).

    Returns:
        (:obj:`dict` of :obj:`dict`): The python dictionary of profiles.
            The member id is a key, a profile of the member is the value.
        
    """
    fname = pobj
    with open(fname,'rb') as f:
        pcrm = pickle.load(f)
    
    return pcrm


def load_social_media_profiles(pobj = './data/twitter_profile.pickle', media='Twitter'):
    """The utility loads the social media profile.

    Args:
        pobj (:obj:`str`): The string pointer to the serialized python dictionary (:obj:`pickle`).

    Returns:
        (:obj:`dict` of :obj:`dict`): The python dictionary of profiles.
            The member id is a key, a profile of the member is the value.
        
    """
    fname = pobj
    with open(fname, 'rb') as f:
        ptwitter = pickle.load(f)
    return ptwitter


def load_behavioral_profile(fname):
    """The utility loads the behavioral paramters from an external file.
    The behavioral paremeters can be preference in expertise seeking such compatibility
    versus similarity in skills; preference on a local partnership versus international, etc.
    
    In short:
        1. partnership preference on skill complementarity.
        2. partnership preference on skill similarity.
        3. partner who is rather within one's own community
        4. partner with whom one already has offline and/ or online interaction 
        5. partner with common motivations such as environmental sustainability, open source.
        
    """
    pass


def load_spirometer_profile(fname):
    """The utility loads the spirometer profile from an external file.
    The input from the community spirals will be combined with the corresponding the behavioral choice (5). 
    
    """
    pass


def load_values_profile(fname):
    """The values based on the Schwartz model. 
    
    """
    pass

In [16]:
crm = load_social_media_profiles()

In [184]:
coMat = load_skill_cooccurance()
net = form_network_relations(coMat)
pp.pprint(net)

{'3d': {'co': {'3dmodel': 1,
            '3dprint': 4,
            'additive': 1,
            'ar': 1,
            'artist': 1,
            'cad': 1,
            'construction': 1,
            'craft': 1,
            'creative': 1,
            'creativity': 1,
            'design': 10,
            'development': 1,
            'digitalfabrication': 3,
            'disability': 1,
            'education': 2,
            'empowering': 1,
            'enabling': 1,
            'engineer': 1,
            'engineering': 2,
            'enterpreneurship': 1,
            'entrepreneur': 2,
            'fablab': 2,
            'health': 1,
            'industry40': 1,
            'innovation': 15,
            'iot': 2,
            'jewellery': 1,
            'lpg': 1,
            'maker': 3,
            'manufacturing': 4,
            'material': 3,
            'metal': 1,
            'open': 2,
            'printer': 1,
            'product': 1,
            'production': 2,
            'proje

             'furniture': 1,
             'innovation': 1,
             'maker': 1,
             'metal': 1,
             'milling': 1,
             'precision': 1,
             'restoration': 1,
             'wood': 1},
       'f': 4},
'cncrouter': {'co': {'3dmodel': 1,
                   'airbrush': 1,
                   'geometry': 1,
                   'iot': 1,
                   'light': 1,
                   'makingthings': 1,
                   'map': 1,
                   'woodworking': 1},
             'f': 2},
'co design': {'co': {'creative': 1,
                   'curiosity': 1,
                   'design': 1,
                   'development': 1,
                   'engineering': 2,
                   'enterpreneurship': 1,
                   'facilitator': 1,
                   'healthcare': 1,
                   'hmi': 1,
                   'industrial': 1,
                   'innovation': 2,
                   'open': 1,
                   'opensource': 1,
              

                    'cooperation': 1,
                    'creative': 1,
                    'creativity': 1,
                    'design': 1,
                    'determined': 1,
                    'education': 1,
                    'laboratory': 1,
                    'people': 1,
                    'raspberrypi': 1,
                    'sensors': 1,
                    'social': 2,
                    'socialinnovation': 1,
                    'sport': 1},
              'f': 6},
'discoverers': {'co': {'designers': 1,
                     'developers': 1,
                     'honest': 1,
                     'opensource': 1},
               'f': 1},
'disruption': {'co': {'business strategy': 1,
                    'industrial design': 1,
                    'partnership': 1,
                    'trust': 1},
              'f': 1},
'diving': {'co': {'infrastructure': 1, 'maintenance': 1, 'sea': 1, 'ship': 1},
          'f': 1},
'diy': {'co': {'community': 1, 'opensource': 1, 'proto

                    'f': 1},
'industry': {'co': {'bussines': 1,
                  'chemisty': 1,
                  'ecology': 1,
                  'innovator': 1,
                  'wool': 1},
            'f': 2},
'industry40': {'co': {'3d': 1,
                    '3dprint': 1,
                    'aec': 1,
                    'automation': 1,
                    'bigdata': 3,
                    'bim': 1,
                    'biology': 1,
                    'collaboration': 1,
                    'consultant': 2,
                    'cooperation': 1,
                    'design': 3,
                    'development': 2,
                    'digital': 2,
                    'digitalfabrication': 2,
                    'economist': 1,
                    'education': 3,
                    'empowering': 1,
                    'energy': 1,
                    'engineering': 5,
                    'enterpreneurship': 1,
                    'entrepreneur': 3,
                    'facilita

'openmaker team member': {'co': {'collaborativeeconomy': 1,
                               'design': 1,
                               'enabling': 1,
                               'eu': 1,
                               'explore': 1,
                               'innovation': 2,
                               'machine learning': 1,
                               'makers': 1,
                               'open': 1,
                               'python': 1,
                               'research and development': 1,
                               'socialinnovation': 1,
                               'sofware development': 1,
                               'systemic': 1},
                         'f': 4},
'opensource': {'co': {'3dprint': 1,
                    'aquaponics': 1,
                    'arduino': 1,
                    'art': 1,
                    'bettereurope': 1,
                    'bigdata': 1,
                    'biology': 1,
                    'co design': 1,

'tires': {'co': {'automotive': 1,
               'innovation': 1,
               'leadership': 1,
               'production': 1},
         'f': 1},
'tooling': {'co': {'precisionpatrs': 1, 'production': 1}, 'f': 1},
'tradition': {'co': {'energy': 1,
                   'hydro': 1,
                   'manufacturing': 1,
                   'research and development': 1},
             'f': 1},
'training': {'co': {'bigdata': 1,
                  'datascience': 1,
                  'design': 1,
                  'entrepreneurship': 1,
                  'innovation': 1,
                  'networking': 1,
                  'sustainability': 1},
            'f': 2},
'transport': {'co': {'cycling': 1, 'environment': 1, 'health': 1, 'mobility': 1},
             'f': 1},
'trust': {'co': {'business strategy': 1,
               'ceo': 1,
               'disruption': 1,
               'empathy': 1,
               'fair play': 1,
               'furniture manufacturer': 1,
               'industrial d

In [163]:
#goes into models.py module

class SocialGraph(object):
    """A graph object which represents intra-community social connections.
    The social graph may contain full information on the community or a part of it.
    Such partial network vision might be lack of information of an actor or subset of it.

    Attributes:
            nodes (:obj:`set`): The list of member nodes in the community.
            notmembers (:obj:`set`): The list of non-member nodes in the community.
            attributes (:obj:`dict`): The node attributes of the members.
                The keys are the node ids, those who are member of the community.
                The values themselves are dictionary items of atttribute and value pairs.
            connections (:obj:`dict`): The egonets of each member in the community.
                The keys are the node ids, those who are member of the community.
                The values themselves are dictionary items. Where each key is the connection type
                and the value is a :obj:`set` of node ids:
                    - 'in': The inward link type, the member nodes who follow the ego.
                    - 'in': The outward link type, the member nodes whom the ego follows.
                    - 'bi': The reciprocated links, the member nodes with whom the ego has a
                    mutual followership.
                    - 'ext': The list of not-member nodes whom the ego follows.
            
    
    Todo:
        - Implement node removal method.
        - Differenatiare inward and outward connection between member and not member nodes.
    

    """

    def __init__(self, profiles = None, media = 'Twitter'):
        """The constructor. 

        Args:
            profiles (:obj:`dict`, optional): A json like dict item where used keys are member ids
            and each value is a :obj:`dict` whose keys in return are.
                - 'TwitterId': twitter ID
                - 'nf' : number of friends
                - 'nfollowers': number of followers
                - 'flist': list of friends' twitter IDs.
            media (:obj:`str`, optional): A descriptor of the source of the connection.
            
        """
        self.nodes = set()
        self.notmembers = set()
        self.attributes = {}
        self.connections = {}
        if profiles: self.extend_net(profiles, media)
    
    def add_isolate(self, aid, ismember = True):
        """Adds an isolate to the graph. 

        Args:
            noid (:obj:`int`): A unique node id.
            ismember (:obj:`bool`): whether the node is a member (default True).
            
        Returns:
            (:obj:`bool`): False if node already exists, otherwise it returns True.
            

        """
        if aid in self.nodes: return False
        # was already added as a member
        if aid in self.notmembers: return False
        # was already added as a non-member
        if not ismember:
            self.notmembers.add(aid)
            return True
        self.nodes.add(aid)
        self.connections.update({aid: {'bi': set(),
                                  'in': set(),
                                  'out': set(),
                                  'ext': set()
                                 }
                                })
        return True
    
    
    def remove_node(self, noid):
        """Not implemented yet.
        
        """
        pass
    
    def update_attributes(self, noid, attributes):
        """Updates the attributes of the member nodes.

        Args:
            noid (:obj:`int`): A unique node id 
            attributes (:obj:`dict`): A dictionary with new attributes.
            
        Returns:
            (:obj:`bool`): Returns True.
            

        """
        if not noid in self.attributes.keys():
            self.attributes[noid] = attributes
            # It is very likely that the node is not in the network
            self.add_isolate(noid)
            return True
        for attr, val in attributes.items():
            self.attributes[noid][attr] = val
        return True
    
    def get_attribute(self, noid, attr):
        """Retrieves the value of the specific attribute of the node.

        Args:
            noid (:obj:`str`): A unique node id 
            attr (:obj:`str`): An attribute.
            
        Returns:
            (:obj:`obj`): If attribute exists.
            None: If attributes does not exist
            

        """
        if not noid in self.attributes.keys(): return
        if not attr in self.attributes[noid].keys(): return   
        return self.attributes[noid][attr]
    
    def get_noid(self, query):
        """Retrieves the nodeid for which the attribute matches.

        Args:
            query (:obj:`tuple`): An attribute and value tuple.
            
        Returns:
            (:obj:`str`): The node id of the first attribute match.
            None: If query doesn't match to any node.
            
        """
        attr,val = query
        print('get_noid:')
        for noid in self.attributes.keys():
            if not attr in self.attributes[noid].keys(): continue
            print('noid: {}, query: {}, actual: {}'.format(noid, val, self.attributes[noid][attr]))
            if self.attributes[noid][attr] == val: return noid
        return
            
    
    def add_link(self, start, end, isbi = False):
        """Adds a link to the social graph.

        Args:
            start (:obj:`str`): A unique node id 
            end (:obj:`str`): A unique node id
            isbi (:obj:`bool`): whether the link is a bidirectional (default False).
            
        Returns:
            (:obj:`bool`): Returns True.
            

        """
        if start in self.nodes and end in self.nodes:
            if isbi:
                self.connections[start]['bi'].add(end)
                self.connections[start]['in'].discard(end)
                self.connections[start]['out'].discard(end)
                self.connections[end]['bi'].add(start)
                self.connections[end]['in'].discard(start)
                self.connections[end]['out'].discard(start)
            elif end in self.connections[start]['in']:
                self.connections[start]['bi'].add(end)
                self.connections[end]['bi'].add(start)
                self.connections[start]['in'].discard(end)
                self.connections[end]['out'].discard(start)
            else:
                self.connections[start]['out'].add(end)
                self.connections[end]['in'].add(start)
        elif start in self.nodes:
            self.connections[start]['ext'].add(end)
            self.notmembers.add(end)
        elif end in self.nodes:
            self.connections[end]['ext'].add(start)
            self.notmembers.add(start)
        else:
            self.notmembers.add(start)
            self.notmembers.add(end)
        return True
    
    
    def remove_link(self, start, end, isbi = False):
        """Not implemented yet.
        
        """
        pass
    
    def extend_net(self, profiles, media = 'Twitter'):
        """Given an external connection data, the method populates
        the SocialGraph instance in a batch mode.
        
        Note:
            - Only a social graph from Twitter data is supported at the moment.
            - nfollowers is missing at the moment.

        Args:
            profiles (:obj:`dict`): A json like dict item where used keys are member ids
            and each value is a :obj:`dict` whose keys in return are
                - 'TwitterId': twitter ID
                - 'nf' : number of friends
                - 'nfollowers': number of followers
                - 'flist': list of friends' twitter IDs.
             media (:obj:`str`, optional): A descriptor of the source of the connection.
            
        Returns:
            (:obj:`bool`): False if node already exists, otherwise it returns True.
            

        """
        mids = profiles.keys()
        for m in mids:
            self.add_isolate(m)
            self.update_attributes(m,
                                   {'TwitterId': int(profiles[m]['TwitterId']),
                                    #'nfollowers': profiles[m]['nfollowers'], #is missing at the moment.
                                    'nfriends': int(profiles[m]['nf'])
                                   }
                                  )
        for from_id in mids:
            for to_id in mids:
                if from_id == to_id: continue
                ifrom_id = int(profiles[from_id]['TwitterId'])
                ito_id = int(profiles[to_id]['TwitterId'])
                afriends = profiles[from_id]['flist']
                bfriends = profiles[to_id]['flist']
                ab = ito_id in afriends
                ba = ifrom_id in bfriends
                if ab and ba: self.add_link(from_id, to_id, isbi=True)
                elif ab: self.add_link(from_id, to_id)
                elif ba: self.add_link(to_id, from_id)
                else: pass
        media_ids = set([v['TwitterId'] for k, v in self.attributes.items()])
        for m in mids:
            friends = set(profiles[m]['flist'])
            ext = friends.difference(media_ids)
            self.connections[m]['ext'] = ext
            self.notmembers = self.notmembers.union(ext)
        return True
    
    
    @staticmethod
    def compare(setA, setB, raw = False, exclusive = True):
        """The method makes a set theoretic comparisons of two input sets.
        
        Note: The order of input sets are important.
        
        Args:
            setA (:obj:`set` or :obj:`list`): The first set.
            setB (:obj:`set` or :obj:`list`): The second set.
            raw (:obj:`bool`): When it is true the sets of intersection and
                differences are returned (default False)
            exclusive (:obj:`bool`): When it is true for the the amplitude of the differences are
                computed where the denominator is the respective sets (default True)
            
        Returns:
            (:obj:`tuple` of 3 :obj:`set`): For raw = True
            (:obj:`tuple` of 3 :obj:`float`): otherwise. The first item represents the size of
                the intersection of two sets with respect to the union set.
        
        """
        if type(setA) is list: setA = set(setA)
        if type(setB) is list: setB = set(setB)
        assert isinstance(setA, set), 'The first input is expected to be a python set or list.'
        assert isinstance(setB, set), 'The second input is expected to be a python set or list.'
        difAB = setA.difference(setB)
        difBA = setB.difference(setA)
        joint = setA.intersection(setB)
        if raw: return joint, difAB, difBA
        nAB = len(setA.union(setB))
        if nAB == 0: return 0,0,0
        nA = len(setA)
        nB = len(setB)
        ndifAB = len(difAB)
        ndifBA = len(difBA)
        njoint = len(joint)
        if exclusive:
            difAB = ndifAB/nA if nA else 0
            difBA = ndifBA/nB if nB else 0
            joint = njoint/nAB if nAB else 0
            return joint, difAB, difBA
        return njoint/nAB, ndifAB/nAB, ndifBA/nAB
    
    
    def get_ego(self, aid):
        """Returns the immediate connection of the node.
        
        Args:
            aid (:obj:`str`): A unique node id 
            
        Returns:
            (:obj:`tuple` of :obj:`int` and :obj:`dict`): The first of the tuple can be either 0 or 1.
                0 denoting that the node was not found, 1 denotes the node exists.
                The dictionary which is the second item in the tuple denotes the connections of the ego
                where whose keys are the connection types.
            
        """
        if aid in self.connections.keys():
            return 1, self.connections[aid]
        else:
            return 0, {'bi': set(),'in': set(),'out': set(),'ext': set()}
        
    
    def compute_influence(self, aid):
        """Returns the network influence of the node.
        
        Note:
            The network influence in this version is a modified in_degree centrality.
            $influence = total in degree + community in degree$ 
            In words, the connection received within the community is counted twice.
            
        Args:
            aid (:obj:`str`): A unique node id 
            
        Returns:
            (:obj:`float`): The influence score of the agent.
            
        """
        nfollowers = self.get_attribute(aid, 'nfollowers')
        if nfollowers is None: nfollowers = 0
        if aid in self.connections.keys():
            ego = self.connections[aid]
            nfollowers += len(ego['in'].union(ego['bi']))
        return nfollowers
    
    
    def get_network(self):
        """Returns the intra-community links.
        
        Args:
        
        Returns:
            (:obj:`dict`): The unidirectional links are returned in a dictionary format. 
                For each node its outgoing is recorded for the edge lists.
    
        """
        return {n:self.connections[n]['bi'].union(self.connections[n]['out']) for n in self.connections.keys()}
 

In [164]:
TWITTER_PROFILES = load_social_media_profiles('./data/twitter_profile.pickle')
SAMPLE = get_sample(TWITTER_PROFILES, 20)
SAMPLE.keys()

dict_keys(['155306256', '153643752', '155241805', '155594430', '155709442', '157496825', '155525266', '155499906', '142249283', '135377966', '157784656', '155222038', '156582037', '153716747', '155473092', '155524507', '155546388', '155521973', '155740116', '157496737'])

In [165]:
#SG = SocialGraph(SAMPLE)
SG = SocialGraph(TWITTER_PROFILES)
print(get_attributes(SG))
print(get_attributes(SocialGraph))

['nodes', 'notmembers', 'attributes', 'connections']
['add_isolate', 'remove_node', 'update_attributes', 'get_attribute', 'get_noid', 'add_link', 'remove_link', 'extend_net', 'compare', 'get_ego', 'compute_influence', 'get_network']


In [166]:
print(len(SG.nodes))
print(len(SG.notmembers))
#pp.pprint(SG.connections)
#print(SG.attributes)

157
75104


In [167]:
help(SG.get_ego)

Help on method get_ego in module __main__:

get_ego(aid) method of __main__.SocialGraph instance
    Returns the immediate connection of the node.
    
    Args:
        aid (:obj:`str`): A unique node id 
        
    Returns:
        (:obj:`tuple` of :obj:`int` and :obj:`dict`): The first of the tuple can be either 0 or 1.
            0 denoting that the node was not found, 1 denotes the node exists.
            The dictionary which is the second item in the tuple denotes the connections of the ego
            where whose keys are the connection types.



In [168]:
help(SG.compute_influence)

Help on method compute_influence in module __main__:

compute_influence(aid) method of __main__.SocialGraph instance
    Returns the network influence of the node.
    
    Note:
        The network influence in this version is a modified in_degree centrality.
        $influence = total in degree + community in degree$ 
        In words, the connection received within the community is counted twice.
        
    Args:
        aid (:obj:`str`): A unique node id 
        
    Returns:
        (:obj:`float`): The influence score of the agent.



In [169]:
for aid in SG.nodes:
    print(aid, SG.compute_influence(aid))

154151913 3
156582037 8
155468589 7
154449529 0
155715549 0
155170222 5
155760737 2
154536472 0
158526065 26
153818672 21
155089214 4
155736202 0
155709442 4
155222567 0
155504513 0
155739856 0
154671114 0
153716747 0
155481818 0
154687545 2
154678051 8
157496687 0
155740257 0
155241805 0
154860115 0
152031440 2
155790083 0
139195444 12
154270933 0
154735576 10
155743075 0
155525266 0
155473092 5
158326587 5
157496690 0
154747461 4
152583764 14
155657933 0
153762636 3
155729610 0
154578841 7
155656873 0
153643752 0
154683766 2
157496725 1
156649741 0
154518289 4
145489262 16
155519905 1
155499906 0
155759507 12
141446497 1
155101022 0
153753368 0
156644216 0
156653842 3
155480169 0
155083636 0
155602135 7
156002767 1
155521973 8
141907540 6
154865718 2
157238215 8
153680906 4
154868197 1
142249277 2
155731312 2
154653833 0
153507756 0
157496726 7
154683832 0
157496825 0
155739139 8
155873479 1
155394915 0
155188374 0
155740116 0
154567552 1
154442156 0
142954818 1
157784656 14
15452410

In [170]:
aid1 = '155714799'
aid2 = '158526065'
aid3 = '153818672'
aid4 = '155739856'

In [171]:
egoA = SG.get_ego(aid1)[1]
egoB = SG.get_ego(aid2)[1]
egoC = SG.get_ego(aid3)[1]
egoD = SG.get_ego(aid4)[1]

In [172]:
print(egoA['in'],egoA['out'],egoA['bi'], len(egoA['ext']))
print(egoB['in'],egoB['out'],egoB['bi'], len(egoB['ext']))
print(egoC['in'],egoC['out'],egoC['bi'], len(egoC['ext']))
print(egoD['in'],egoD['out'],egoD['bi'], len(egoD['ext']))

{'157784656', '156612789', '157496826'} {'157496726', '156653842'} {'135377966', '140411756', '153818672', '145489262', '153862027', '153757086', '152583764', '140411415', '156582037', '152031440', '152730563', '154518289'} 304
{'154567552', '154151913', '154683766', '155468589', '153941328', '153838985', '155486371', '155259995', '153818672', '158307955', '153862027', '155709442', '154993383', '155594430', '154687545', '155148177', '155521973', '155103070', '153689099', '155742649', '154653833', '155085145', '155743739', '153750682', '155473092', '155739139'} set() set() 392
{'158307955', '153862027', '154678051', '155343559', '140411415', '154865718', '153680906', '155742649', '152583764'} {'155731312', '158526065', '155668791', '154578841', '157496826', '155486371', '155660568', '154518289'} {'135377966', '140411756', '145489262', '154735576', '155722638', '154993383', '155222038', '154670192', '155714799', '157784656', '156653842', '153941328'} 2770
set() set() set() 111


In [173]:
help(SG.compare)

Help on function compare in module __main__:

compare(setA, setB, raw=False, exclusive=True)
    The method makes a set theoretic comparisons of two input sets.
    
    Note: The order of input sets are important.
    
    Args:
        setA (:obj:`set` or :obj:`list`): The first set.
        setB (:obj:`set` or :obj:`list`): The second set.
        raw (:obj:`bool`): When it is true the sets of intersection and
            differences are returned (default False)
        exclusive (:obj:`bool`): When it is true for the the amplitude of the differences are
            computed where the denominator is the respective sets (default True)
        
    Returns:
        (:obj:`tuple` of 3 :obj:`set`): For raw = True
        (:obj:`tuple` of 3 :obj:`float`): otherwise. The first item represents the size of
            the intersection of two sets with respect to the union set.



In [174]:
SG.compare(egoA['bi'], egoC['bi'])

(0.14285714285714285, 0.75, 0.75)

In [175]:
help(SG.get_network)

Help on method get_network in module __main__:

get_network() method of __main__.SocialGraph instance
    Returns the intra-community links.
    
    Args:
    
    Returns:
        (:obj:`dict`): The unidirectional links are returned in a dictionary format. 
            For each node its outgoing is recorded for the edge lists.



In [176]:
SG.get_network()

{'135377966': {'135574317',
  '139195444',
  '140411415',
  '140411756',
  '145489262',
  '152583764',
  '152730563',
  '153562611',
  '153689099',
  '153757086',
  '153762636',
  '153818672',
  '153862027',
  '154578841',
  '154678051',
  '154735576',
  '154747461',
  '154865718',
  '155546218',
  '155714799',
  '156582037',
  '156612789',
  '157238215',
  '157496726',
  '157784656'},
 '135574317': set(),
 '139195439': {'145489262', '154678051', '154735576', '157238215'},
 '139195444': {'135377966',
  '141907540',
  '142249283',
  '145489262',
  '152583764',
  '153702260',
  '153762329',
  '155399752',
  '155668791',
  '155710159',
  '155726182',
  '155735454',
  '155759507'},
 '140411415': {'135377966',
  '140411756',
  '145489262',
  '152031440',
  '152583764',
  '152730563',
  '153689099',
  '153757086',
  '153818672',
  '153862027',
  '154747461',
  '155714799',
  '156582037',
  '156612789',
  '157784656'},
 '140411418': {'153689099'},
 '140411756': {'135377966',
  '140411415',
  

In [177]:
help(SocialGraph.add_link)

Help on function add_link in module __main__:

add_link(self, start, end, isbi=False)
    Adds a link to the social graph.
    
    Args:
        start (:obj:`str`): A unique node id 
        end (:obj:`str`): A unique node id
        isbi (:obj:`bool`): whether the link is a bidirectional (default False).
        
    Returns:
        (:obj:`bool`): Returns True.



In [178]:
SG.add_link(aid2, aid3, isbi = True)

True

In [179]:
SG.get_network()

{'135377966': {'135574317',
  '139195444',
  '140411415',
  '140411756',
  '145489262',
  '152583764',
  '152730563',
  '153562611',
  '153689099',
  '153757086',
  '153762636',
  '153818672',
  '153862027',
  '154578841',
  '154678051',
  '154735576',
  '154747461',
  '154865718',
  '155546218',
  '155714799',
  '156582037',
  '156612789',
  '157238215',
  '157496726',
  '157784656'},
 '135574317': set(),
 '139195439': {'145489262', '154678051', '154735576', '157238215'},
 '139195444': {'135377966',
  '141907540',
  '142249283',
  '145489262',
  '152583764',
  '153702260',
  '153762329',
  '155399752',
  '155668791',
  '155710159',
  '155726182',
  '155735454',
  '155759507'},
 '140411415': {'135377966',
  '140411756',
  '145489262',
  '152031440',
  '152583764',
  '152730563',
  '153689099',
  '153757086',
  '153818672',
  '153862027',
  '154747461',
  '155714799',
  '156582037',
  '156612789',
  '157784656'},
 '140411418': {'153689099'},
 '140411756': {'135377966',
  '140411415',
  

In [180]:
help(SG.get_attribute)

Help on method get_attribute in module __main__:

get_attribute(noid, attr) method of __main__.SocialGraph instance
    Retrieves the value of the specific attribute of the node.
    
    Args:
        noid (:obj:`str`): A unique node id 
        attr (:obj:`str`): An attribute.
        
    Returns:
        (:obj:`obj`): If attribute exists.
        None: If attributes does not exist



In [181]:
SG.compare(egoA['bi'], egoC['bi'])

(0.13636363636363635, 0.75, 0.7692307692307693)

In [200]:
#goes into models.py module
class CognitiveExpertise(object):
    """A python object that contains methods and data models where an agent, be an indivudual or 
        a community, constructs and adopts its perception of expertise around his environment.

    Attributes:
        nproj (:obj:`int`): The number of projects knowledge that are used to form mainly the skill complementarity
            model.
        nprof (:obj:`int`): The number of member profiles that are used to form mainly the skill similarity 
            model of the community members.
        tags_proj (:obj:`list` of :obj:`str`): Holds the list of skills(column names) extracted from projects.
        tags_prof (:obj:`list` of :obj:`str`): Holds the list of skills(column names) extracted from profiles.
        mat_proj (:obj:`dict`): A dictionary whose values are dictionary holds the frequency counts 
            of the co-occurances in projects. The keys are skills. Ex: {skilla:{skillb:f, skillc:f ...} ...}
        mat_prof (:obj:`dict`): A dictionary whose values are dictionary holds the frequency counts 
            of the co-occurances on profiles. The keys are skills. Ex: {skilla:{skillb:f, skillc:f ...} ...}
        tf_proj (:obj:`dict`): A dictionary whose keys are the skills observed in projects and
            values are the occurance counts.
        tf_prof (:obj:`dict`): A dictionary whose keys are the skills observed on member profiles and 
            values are the occurance counts.
        centralities (:obj:`dict`): If computed, holds the centrality measure of each skill as of their 
            occurance on projects. The more often a skill is required used in a project the higher
            centrality score it will get.
        diffusions (:obj:`dict`): If computed, holds the diffusion measure of each skill as of their 
            occurance on member profiles. The more commen a skill is to the community the higher
            diffusion score it will get.
        distinctivenesses (:obj:`dict`): If computed, holds the distinctiveness measure of each skill. 
            If a central skill is owned by fewer members to the community, more distinctive is the skill
            for the community. In other words, a skill with high skill and low diffusion value will get
            higher distinctiveness value.

    """

    def __init__(self, projects, profiles):
        """The constructor.
        
        Args:
            projects (:obj:`list` of :obj:`set` of :obj:`str`): List of skill co-occurances in projects.
            profiles (:obj:`list` of :obj:`set` of :obj:`str`): List of skill co-occurances on profiles.
        
        """
        self.nproj = len(projects)
        self.nprof = len(profiles)
      
        netproj = form_network_relations(projects)
        netprof = form_network_relations(profiles)
        
        self.tags_proj = list(netproj.keys())
        self.tags_prof = list(netprof.keys())
        
        self.tf_proj = {t:netproj[t]['f'] for t in netproj.keys()}
        self.tf_prof = {t:netprof[t]['f'] for t in netprof.keys()}
                
        self.mat_proj = {t:netproj[t]['co'] for t in netproj.keys()}
        self.mat_prof = {t:netprof[t]['co'] for t in netprof.keys()}
    
    
    def _get_cooccurances(self, a, b, tags, mat):
        """A given two skills it returns co-occurance of them.

        Args:
            a (:obj:`str`): A skill identifier
            b (:obj:`str`): Another skill identfier.
            tags: (:obj:`list` of :obj:`str`): Holds the list of skills(column names),
            mat (:obj:`dict`): A dictionary whose values are dictionary holds
                the frequency counts.

        Returns:
            None: When either a or b is not occured.
            (:obj:`int`): The raw co-occurance frequency.

        """
        if a not in tags: return
        if b not in tags: return
        if b not in mat[a].keys(): return 0
        return mat[a][b]
    
    
    def get_profile_cooccurances(self, a, b):
        """A given two skills it returns co-occurance of them in profiles.

        Args:
            a (:obj:`str`): A skill identifier
            b (:obj:`str`): Another skill identfier.

        Returns:
            None: When either a or b is not occured.
            (:obj:`int`): The raw co-occurance frequency.

        """
        return self._get_cooccurances(a, b,
                                      self.tags_prof,
                                     self.mat_prof) 
    
    
    def get_project_cooccurances(self, a, b):
        """A given two skills it returns co-occurance of them in projects.

        Args:
            a (:obj:`str`): A skill identifier
            b (:obj:`str`): Another skill identfier.

        Returns:
            None: When either a or b is not occured.
            (:obj:`int`): The raw co-occurance frequency.

        """
        return self._get_cooccurances(a, b,
                                      self.tags_proj,
                                      self.mat_proj) 

    
    def compute_centralities(self, raw=True):
        """Computes the centralities of skillset that occurance matrix.
        
        Note: 
            This is a mapping of skills on a co-occurance map: the more often a skill is required
            for the projects in a community the more central is the skill to the community.
            
        Args:
            raw (:obj:`bool`): Is used to determine whether raw count or normalized values should be returned
            (default True).

        Returns:
            (:obj:`dict`): A dictionary holding the centrality of each skill.
            
        """
        self.centralities = dict()
        for t in self.tags_proj:
            self.centralities[t] = self.get_centrality(t,raw)
        return self.centralities
    

    def get_centrality(self, a, raw=True):
        """Returns the frequency count of a skill in co-occurance matrix formed from project requirements.
        
        Note: 
            As of the co-occurance map: the more often a skill is required for the projects in a community
            the more central is the skill to the community.
        Args:
            a (:obj:`str`): A skill identifier
            raw (:obj:`bool`): Is used whether raw count needs to be returned (default True).

        Returns:
            None: When no project info has found.
            (:obj:`int`): The raw co-occurance frequency when arg raw = True.
            (:obj:`float`): The normalized frequency count as of total project observations.
            
        """
        if a not in self.tags_proj: return 0
        if not self.nproj: return
        fa = self.tf_proj[a]
        if raw: return fa
        return fa / (1.0 * self.nproj)

    
    def compute_diffusions(self, raw=True):
        """Computes the diffusion of skills among the community members.
        
        Args:
            raw (:obj:`bool`): Is used to determine whether raw count or normalized values should be returned
            (default True).

        Returns:
            (:obj:`dict`): A dictionary holding the diffusion score for each skill.
            
        """
        self.diffusions = dict()
        for t in self.tags_prof:
            self.diffusions[t] = self.get_diffusion(t,raw)
        return self.diffusions

    
    def get_diffusion(self, a, raw=True):
        """Returns the frequency count of a skill in co-occurance matrix formed from agent profiles.

        Note: 
            As of the co-occurance map: the more often a skill is observed within a community the more
            diffused is the skill in the community.
            
        Args:
            a (:obj:`str`): A skill identifier
            raw (:obj:`bool`): Is used to determine whether raw count needs to be returned (default True).

        Returns:
            None: When no profile info has found.
            (:obj:`int`): The raw co-occurance frequency when the arg raw = True.
            (:obj:`float`): The normalized frequency count as of $total profiles/members$ in the community.
            
        """
        if a not in self.tags_prof: return 0
        if not self.nprof: return
        
        fa = self.tf_prof[a]
        if raw: return fa
        return fa / (1.0 * self.nprof)

    
    def compute_distinctivenesses(self):
        """Computes the distinctiveness of skills within the community parctices.
        
        Returns:
            (:obj:`dict`): A dictionary holding the distinctiveness score for each skill.
            
        """
        self.distinctivenesses = dict()
        for t in self.tags_proj:
            self.distinctivenesses[t] = self.get_distinctiveness(t)
        return self.distinctivenesses
    
    
    def get_distinctiveness(self, a):
        """Returns a score to measure to what extend an important skill for the community is owned by less
            members.
            
        Args:
            a (:obj:`str`): A skill identifier
            
        Returns:
            None: When no profile info has found.
            (:obj:`int`): The raw co-occurance frequency when the arg raw = True.
            (:obj:`float`): The normalized frequency count as of total profiles/members in the community.
            
        """
        x = self.get_diffusion(a)
        y = self.get_centrality(a)
        if not x: return
        if y is None: return 
        return y / x


    def get_union(self, a, b, source='projects'):
        """Returns total occurances where a or b is observed.
        
        Args:
            a (:obj:`str`): A skill identifier
            b (:obj:`str`): Another skill identfier.
            source (:obj:`str`): Denotes the type of data source.
                It can be either 'projects' or 'profiles' (default 'projects')

        Returns:
            None: When either a or b is not occured.
            (:obj:`int`): The magnitude of the frquency counts where either a or b has occured.

        """
        if source == 'profiles':
            fa = self.tf_prof[a]
            fb = self.tf_prof[b]
            fab = self.get_project_cooccurances(a,b)
        else:
            fa = self.tf_proj[a]
            fb = self.tf_proj[b]
            fab = self.get_profile_cooccurances(a,b)
        return fa + fb - fab

    
    def get_difference(self, a, b, raw=True, exclusive=True, source='projects'):
        """Returns the count of occurances of a without b.
        
        Args:
            a (:obj:`str`): A skill identifier
            b (:obj:`str`): Another skill identfier.
            raw (:obj:`bool`): whether plain difference counts to be returned (default True).
            exclusive (:obj:`bool`): whether difference is normalized by size of the own set
                or the union set (default True).
            source (:obj:`str`): Denotes the type of data source. It can be either 
                'projects' or 'profiles' (default 'projects')

        Returns:
            None: When either a or b is not occured.
            (:obj:`int`): The occurance frquency counts of a without b 
                when a raw count is requested.
            (:obj:`float`): The ratio of occurance of a without b with respect to frequencies of a
                or with resepct to cases where either frequencies of a or b has been observed.

        """
        if source == 'projects':
            fa = self.tf_proj[a]
            fab = self.get_project_cooccurances(a,b)
        else:
            fa = self.tf_prof[a]
            fab = self.get_profile_cooccurances(a,b)    
        dif = fa - fab
        if raw: return dif
        if exclusive: return dif / (1.0 * fa)
        return dif / (1.0 * self.get_union(a,b,source))

    
    def get_similarity(self, a,b, raw=True, exclusive=True, source='projects'):
        """Returns the count of co-occurances of a and b.

        Note: When the union is used as the the reference, that is when exclusive is false,
            sim(a,b) = sim(b,a)
            
        Args:
            a (:obj:`str`): A skill identifier
            b (:obj:`str`): Another skill identfier.
            source (:obj:`str`): Denotes the type of data source. It can be either 
                'projects' or 'profiles' (default 'projects')

        Returns:
            None: When either a or b is not occured.
            (:obj:`int`): The joint occurances of a and b.

        """
        if source == 'projects':
            fab = self.get_project_cooccurances(a,b)
            fa = self.tf_proj[a]
        else:
            fab = self.get_profile_cooccurances(a,b)
            fa = self.tf_prof[a]   
        if raw: return fab
        if exclusive: return fab / (1.0 * fa)
        return fab / (1.0 * self.get_union(a,b,source))

    
    def get_resemblance(self, a, b, source='projects'):
        """Returns the resemblance of a and b in terms of both their similarities and differences.
        
        Args:
            a (:obj:`str`): A skill identifier
            b (:obj:`str`): Another skill identfier.
            source (:obj:`str`): Denotes the type of data source. It can be either 
                'projects' or 'profiles' (default 'projects')

        Returns:
            None: When either a or b is not occured.
            (:obj:`float`): The resemblance ratio.

        """
        if source == 'projects':
            fab = self.get_project_cooccurances(a,b)
        else:
            fab = self.get_profile_cooccurances(a,b)    
        dif_ab = self.get_difference(a,b,raw=False,source=source)
        dif_ba = self.get_difference(b,a,raw=False,source=source)
        sim_ab = self.get_similarity(a,b,raw=False,exclusive=False,source=source)
        return sim_ab * (1 - abs(dif_ab - dif_ba))

    
    def get_compatibility(self, a, b, source='projects'):
        """Returns a score to measure the compatibility of `a` to `b`, which is,
            given all occurances of skill `b` it measures to what extend skill `a` cooccurs
            along with skill `b`.

        Note: If, for instance, skill b happens to be occuring only when skill `a` occurs
            then the compatibility of `a` to `b` would be 1. In an asymmetric compatibility
            (that is `a` to `b` VS  `b` to `a`) where one of the compatibilities is almost 1 or 1,
            this may point out a  skill and sub-skill relation.
        
        Args:
            a (:obj:`str`): A skill identifier
            b (:obj:`str`): Another skill identfier.
            source (:obj:`str`): Denotes the type of data source. It can be either 
                'projects' or 'profiles' (default 'projects')

        Returns:
            None: When either a or b is not occured.
            (:obj:`float`): The compatibility score of skill `a` to skill `b`.

        """
        return self.get_similarity(b, a, raw=False, exclusive=True,source=source)
    
  
    def update_relevance_models(self, new_info):
        """Given new external project information, the module updates the data model
            that is used to compute expertise relations for an agent or for a community.
        
        Args:
            new_info is one of
                (:obj:`list` of :obj:`str`): A list of skills that appeared on a project requirement.
                (:obj:`list` of :obj:`list` of :obj:`str`): A list of skill co-occurances.

        Returns:
            (:obj:`bool`): Notifying the completion of learning.
        
        """
        if not new_info: return
        if type(new_info[0]) is list: 
            new_projects = [set(s) for s in new_info]
        else:
            new_projects = [set(new_info)]
        for aset in new_projects:
            self.nproj += 1 
            for a in aset:
                if a not in self.tags_proj:
                    self.tags_proj.append(a)
                    self.tf_proj[a] = 0
                    self.mat_proj[a] = {}
                self.tf_proj[a] += 1
                for b in aset:
                    if a == b: continue
                    if b in self.mat_proj[a].keys():
                        self.mat_proj[a][b] += 1
                    else:
                        self.mat_proj[a][b] = 1
        self.compute_centralities()
        self.compute_distinctivenesses()
        return True
        
    
    def update_diffusion_levels(self, new_profiles):
        """Given new profile information, the module updates the data model
            that is used to compute diffusion of skills in the community or
            around the ego network of an agent.
            
        Note: The compuation of this behaviour and the behaviour regarding skill relations 
            update is very similar, where a computational abstraction via paramterization may be
            considered at a final iteration. 
        
        Args:
            new_prfiles is one of
                (:obj:`list` of :obj:`str`): A list of skills that appeared on a prosile.
                (:obj:`list` of :obj:`list` of :obj:`str`): A list of skill co-occurances on profiles.

        Returns:
            (:obj:`bool`): Notifying the completion of update on the skill carriers.
        
        """
        
        if not new_profiles: return
        if type(new_profiles[0]) is list: 
            new_profiles = [set(s) for s in new_profiles]
        else:
            new_profiles = [set(new_profiles)]
        for aset in new_profiles:
            self.nprof += 1 
            for a in aset:
                if a not in self.tags_prof:
                    self.tags_prof.append(a)
                    self.tf_prof[a] = 0
                    self.mat_prof[a] = {}
                self.tf_prof[a] += 1
                for b in aset:
                    if a == b: continue
                    if b in self.mat_prof[a].keys():
                        self.mat_prof[a][b] += 1
                    else:
                        self.mat_prof[a][b] = 1
        self.compute_diffusions()
        self.compute_distinctivenesses()
        return True
    

In [201]:
help(CognitiveExpertise)

Help on class CognitiveExpertise in module __main__:

class CognitiveExpertise(builtins.object)
 |  A python object that contains methods and data models where an agent, be an indivudual or 
 |      a community, constructs and adopts its perception of expertise around his environment.
 |  
 |  Attributes:
 |      nproj (:obj:`int`): The number of projects knowledge that are used to form mainly the skill complementarity
 |          model.
 |      nprof (:obj:`int`): The number of member profiles that are used to form mainly the skill similarity 
 |          model of the community members.
 |      tags_proj (:obj:`list` of :obj:`str`): Holds the list of skills(column names) extracted from projects.
 |      tags_prof (:obj:`list` of :obj:`str`): Holds the list of skills(column names) extracted from profiles.
 |      mat_proj (:obj:`dict`): A dictionary whose values are dictionary holds the frequency counts 
 |          of the co-occurances in projects. The keys are skills. Ex: {skilla:{sk

In [183]:
get_attributes(CognitiveExpertise)

['get_profile_cooccurances',
 'get_project_cooccurances',
 'compute_centralities',
 'get_centrality',
 'compute_diffusions',
 'get_diffusion',
 'compute_distinctivenesses',
 'get_distinctiveness',
 'get_union',
 'get_difference',
 'get_similarity',
 'get_resemblance',
 'get_compatibility',
 'update_relevance_models',
 'update_diffusion_levels']

In [190]:
coMat = load_skill_cooccurance()
CE = CognitiveExpertise(coMat,coMat)
#pp.pprint(coMat)

In [191]:
get_attributes(CE)

['nproj',
 'nprof',
 'tags_proj',
 'tags_prof',
 'tf_proj',
 'tf_prof',
 'mat_proj',
 'mat_prof']

In [187]:
CE.mat_prof

{'3d': {'3dmodel': 1,
  '3dprint': 4,
  'additive': 1,
  'ar': 1,
  'artist': 1,
  'cad': 1,
  'construction': 1,
  'craft': 1,
  'creative': 1,
  'creativity': 1,
  'design': 10,
  'development': 1,
  'digitalfabrication': 3,
  'disability': 1,
  'education': 2,
  'empowering': 1,
  'enabling': 1,
  'engineer': 1,
  'engineering': 2,
  'enterpreneurship': 1,
  'entrepreneur': 2,
  'fablab': 2,
  'health': 1,
  'industry40': 1,
  'innovation': 15,
  'iot': 2,
  'jewellery': 1,
  'lpg': 1,
  'maker': 3,
  'manufacturing': 4,
  'material': 3,
  'metal': 1,
  'open': 2,
  'printer': 1,
  'product': 1,
  'production': 2,
  'project': 2,
  'prototypes': 2,
  'quality': 1,
  'research': 2,
  'research and development': 3,
  'robotics': 1,
  'sales': 1,
  'share': 2,
  'slicer': 1,
  'slm': 1,
  'smart': 1,
  'social': 2,
  'textile': 1,
  'vr': 1},
 '3dmodel': {'3d': 1,
  '3dprint': 2,
  '3dscanning': 1,
  'airbrush': 1,
  'architechture': 1,
  'cad': 1,
  'cncrouter': 1,
  'creativity': 1,


In [188]:
CE.tf_prof

{'3d': 28,
 '3dmodel': 5,
 '3dp': 1,
 '3dprint': 23,
 '3dscanning': 3,
 'accessibility': 1,
 'accessibilità': 1,
 'additive': 2,
 'aec': 1,
 'aeronautics': 1,
 'aerospace': 1,
 'agile': 2,
 'agrotourism': 1,
 'ai': 1,
 'airbrush': 1,
 'anthropocene': 2,
 'applied': 1,
 'aquaponics': 1,
 'ar': 1,
 'architechture': 3,
 'architect': 3,
 'arduino': 6,
 'art': 12,
 'arte': 1,
 'artist': 8,
 'audio': 1,
 'automation': 4,
 'automotive': 2,
 'awareness': 2,
 'beauty': 1,
 'bellezza': 2,
 'bettereurope': 1,
 'biesse': 1,
 'bigdata': 8,
 'bim': 1,
 'biohacker': 1,
 'biological': 2,
 'biology': 4,
 'board': 1,
 'bottle': 1,
 'building': 1,
 'business': 2,
 'business strategy': 2,
 'bussines': 1,
 'cad': 4,
 'ceo': 1,
 'ceramics': 1,
 'cfd': 1,
 'changemaker': 1,
 'chemisty': 1,
 'childrens': 1,
 'ciao': 1,
 'cities': 2,
 'citizens': 2,
 'cloud': 1,
 'cnc': 4,
 'cncrouter': 2,
 'co design': 6,
 'coach': 1,
 'colaboration': 1,
 'collaboration': 4,
 'collaborativeeconomy': 5,
 'collaborator': 7,
 'c

In [192]:
CE.compute_centralities()

{'3d': 28,
 '3dmodel': 5,
 '3dp': 1,
 '3dprint': 23,
 '3dscanning': 3,
 'accessibility': 1,
 'accessibilità': 1,
 'additive': 2,
 'aec': 1,
 'aeronautics': 1,
 'aerospace': 1,
 'agile': 2,
 'agrotourism': 1,
 'ai': 1,
 'airbrush': 1,
 'anthropocene': 2,
 'applied': 1,
 'aquaponics': 1,
 'ar': 1,
 'architechture': 3,
 'architect': 3,
 'arduino': 6,
 'art': 12,
 'arte': 1,
 'artist': 8,
 'audio': 1,
 'automation': 4,
 'automotive': 2,
 'awareness': 2,
 'beauty': 1,
 'bellezza': 2,
 'bettereurope': 1,
 'biesse': 1,
 'bigdata': 8,
 'bim': 1,
 'biohacker': 1,
 'biological': 2,
 'biology': 4,
 'board': 1,
 'bottle': 1,
 'building': 1,
 'business': 2,
 'business strategy': 2,
 'bussines': 1,
 'cad': 4,
 'ceo': 1,
 'ceramics': 1,
 'cfd': 1,
 'changemaker': 1,
 'chemisty': 1,
 'childrens': 1,
 'ciao': 1,
 'cities': 2,
 'citizens': 2,
 'cloud': 1,
 'cnc': 4,
 'cncrouter': 2,
 'co design': 6,
 'coach': 1,
 'colaboration': 1,
 'collaboration': 4,
 'collaborativeeconomy': 5,
 'collaborator': 7,
 'c

In [196]:
CE.compute_diffusions()

{'3d': 28,
 '3dmodel': 5,
 '3dp': 1,
 '3dprint': 23,
 '3dscanning': 3,
 'accessibility': 1,
 'accessibilità': 1,
 'additive': 2,
 'aec': 1,
 'aeronautics': 1,
 'aerospace': 1,
 'agile': 2,
 'agrotourism': 1,
 'ai': 1,
 'airbrush': 1,
 'anthropocene': 2,
 'applied': 1,
 'aquaponics': 1,
 'ar': 1,
 'architechture': 3,
 'architect': 3,
 'arduino': 6,
 'art': 12,
 'arte': 1,
 'artist': 8,
 'audio': 1,
 'automation': 4,
 'automotive': 2,
 'awareness': 2,
 'beauty': 1,
 'bellezza': 2,
 'bettereurope': 1,
 'biesse': 1,
 'bigdata': 8,
 'bim': 1,
 'biohacker': 1,
 'biological': 2,
 'biology': 4,
 'board': 1,
 'bottle': 1,
 'building': 1,
 'business': 2,
 'business strategy': 2,
 'bussines': 1,
 'cad': 4,
 'ceo': 1,
 'ceramics': 1,
 'cfd': 1,
 'changemaker': 1,
 'chemisty': 1,
 'childrens': 1,
 'ciao': 1,
 'cities': 2,
 'citizens': 2,
 'cloud': 1,
 'cnc': 4,
 'cncrouter': 2,
 'co design': 6,
 'coach': 1,
 'colaboration': 1,
 'collaboration': 4,
 'collaborativeeconomy': 5,
 'collaborator': 7,
 'c

In [198]:
CE.compute_distinctivenesses()

{'3d': 1.0,
 '3dmodel': 1.0,
 '3dp': 1.0,
 '3dprint': 1.0,
 '3dscanning': 1.0,
 'accessibility': 1.0,
 'accessibilità': 1.0,
 'additive': 1.0,
 'aec': 1.0,
 'aeronautics': 1.0,
 'aerospace': 1.0,
 'agile': 1.0,
 'agrotourism': 1.0,
 'ai': 1.0,
 'airbrush': 1.0,
 'anthropocene': 1.0,
 'applied': 1.0,
 'aquaponics': 1.0,
 'ar': 1.0,
 'architechture': 1.0,
 'architect': 1.0,
 'arduino': 1.0,
 'art': 1.0,
 'arte': 1.0,
 'artist': 1.0,
 'audio': 1.0,
 'automation': 1.0,
 'automotive': 1.0,
 'awareness': 1.0,
 'beauty': 1.0,
 'bellezza': 1.0,
 'bettereurope': 1.0,
 'biesse': 1.0,
 'bigdata': 1.0,
 'bim': 1.0,
 'biohacker': 1.0,
 'biological': 1.0,
 'biology': 1.0,
 'board': 1.0,
 'bottle': 1.0,
 'building': 1.0,
 'business': 1.0,
 'business strategy': 1.0,
 'bussines': 1.0,
 'cad': 1.0,
 'ceo': 1.0,
 'ceramics': 1.0,
 'cfd': 1.0,
 'changemaker': 1.0,
 'chemisty': 1.0,
 'childrens': 1.0,
 'ciao': 1.0,
 'cities': 1.0,
 'citizens': 1.0,
 'cloud': 1.0,
 'cnc': 1.0,
 'cncrouter': 1.0,
 'co design

In [199]:
get_attributes(CE)

['nproj',
 'nprof',
 'tags_proj',
 'tags_prof',
 'tf_proj',
 'tf_prof',
 'mat_proj',
 'mat_prof',
 'centralities',
 'diffusions',
 'distinctivenesses']

In [127]:
# agents.py
from six import string_types

class Agent(object):
    """A generic agent class.

    Args:
        aid (:obj:`str`): A unique identifier for the agent.
        state (:obj:`str`, optional): Current state of the agent (default 'idle').
        atype (:obj:`str`, optional): The agent type (default 'generic').

    Attributes:
        id (:obj:`str`): The unique identifier for the instance.
        state (:obj:`str`): The current state of the instance.
        type (:obj:`str`): The agent type of the instance.

    """

    def __init__(self, aid, state = 'idle', atype = 'generic'):
        self.id = aid
        self.state = state
        self.type = atype

        
    def current_state(self):
        """It reports the current state of the agent.

        Returns:
            (:obj:`dict`): A `dict` summarizing the state of the agent.

        """
        state = {'id': self.id, 'state': self.state, 'type': self.type}
        return state
    


class Maker(Agent):
    """A maker (:obj:`Agent`) class.

        Notes:

        Attributes:
            name (:obj:`str`, optional): Name of the maker.
            surname (:obj:`str`, optional): Surname of the maker.
            twitter (:obj:`str`, optional): Twitter name of the maker.
            omid (:obj:`int`, optional): OpenMaker platform id of the maker.
            location (:obj:`str`, optional): The geogrpahic location of the maker.
            entity (:obj:`str`, optional): Whether the agent is one of organization 'O' or person 'P'.
            gender (:obj:`str`, optional): The gender is one of 'M', 'F', and 'X.
            tags (:obj:`set` of :obj:`str`, optional): Self assigned tags of the maker.
            skills (:obj:`set` of :obj:`str`, optional): Skills of the agent.
            technologies(:obj:`set` of :obj:`str`, optional): Technologies used by the agent.
            areas (:obj:`set` of :obj:`str`, optional): Interest areas in terms technologies.
            domains (:obj:`set` of :obj:`str`, optional): Activity domains of the agent.

    """
    name = ''
    surname = ''
    twitter = ''
    entity = ''
    location = ''
    gender = ''
    tags = set()
    skills = set()
    technologies = set()
    areas = set()
    domains = set()

    def __init__(self, aid, expertise_model, connections = None, state='idle'):
        super().__init__(aid, state, atype = 'Maker')
        self.modCogExp =  expertise_model
        self.egonet = None
        aid == self.id
        if connections:
            assert isinstance(connections, SocialGraph), 'The connections must be a SocialGraph.'
            self.community = connections
            self.egonet = connections.get_ego(self.id)[1]
        else:
            self.community = SocialGraph() 
            

    def load_profile(self, profile):
        """Updates or loads the profile od the agent.

            Args:
                profile (:obj:`dict`): A python dictionary where keys are attributes of
                this class data attributes and values are the values to be loaded.

            Returns:
                (obj:`bool`): when successful.

        """
        attrs = get_attributes(self)
        attrs.extend(get_attributes(Maker))
        for k,v in profile.items():
            if k not in attrs: continue
            if isinstance(v, string_types): v='"'+str(v)+'"'
            exec("self.{} = {}".format(k,v))

        return True
    
    def skill_similarity(self, skills):
        """Agent's own declared skills to the given an external skill set, agent
            measures the similarity of the given profile to her profile using the knowledge 
            the agent has about the profiles in the community as a reference.
        
        Note: It should be noted that the shared common skills as well as resemblance of differing
            skillsets is added to the score.
            
        Args:
            skills (:obj:`list` of :obj:`str`): The skill set of the other agent.
            
        Returns:
            (:obj:`float`): A similarity score.
            
        """
        f = self.modCogExp.get_resemblance
        # Agent uses its own version of expertise model.
        theother = set(skills)
        common = self.skills.intersection(theother)
        dif_other = theother.difference(common)
        dif_self = self.skills.difference(common)
        lo = list(dif_other) * len(dif_self)
        ls = np.repeat(list(dif_self), len(dif_other))
        nc = len(common)
        return reduce(lambda x,y: x + y if x else y, map(lambda x,y: f(x,y,source='profiles'),lo,ls), nc)
        
    
    def skill_compatibility(self, skills):
        """Comparing agent's own declared skills to the given an external profile, agent
            measure the compatibility of the external profile to her profile considering
            the knowledge the agent has about the projects in the community.
        
        Args:
            skills (:obj:`list` of :obj:`str`): The skill set of the other agent.
            
        Returns:
            (:obj:`float`): The compatibility score.
            
        """
        f = self.modCogExp.get_compatibility
        theother = set(skills)
        common = self.skills.intersection(theother)
        dif_other = theother.difference(common)
        dif_self = self.skills.difference(common)
        lo = list(dif_other) * len(dif_self)
        ls = np.repeat(list(dif_self), len(dif_other))
        return reduce(lambda x,y: x + y if x else y, map(lambda x,y: f(x,y), lo, ls), len(common)) 
            
    
    def skill_difference(self, skills):
        """Agent's compares her own skills to the given an external profile measuring
            the difference of the external profile to her profile considering the knowledge the agent has
            about the projects in the community.
        
        Note: In this implementation for a skill that the other has and the self doesn't, the distincness of
            that skill in the community is used as to weight the differing value of that skill.
        
        Args:
            skills (:obj:`list` of :obj:`str`): The skill set of the other agent.
            
        Returns:
            (:obj:`float`): The skill difference score.
            
        """
        theother = set(skills)
        common = self.skills.intersection(theother)
        dif_other = theother.difference(common) 
        score = 0
        for o in dif_other:
            score_o = 0
            for s in self.skills:
                score_o += self.modCogExp.get_difference(o,s,raw=False,exclusive=False)
            w_o = self.modCogExp.get_distinctiveness(o)
            score += w_o * score_o
        return score

    
    def social_resemblance(self, egonet):
        """Agent compares her social connections to the social connections of another agent. She measures her 
        structural equivalance to the other agent, both in terms of similarities as well as differences.
        
        Note:
            The algorithm below puts a positive bias at being on the same k-cliques for k > 2 as contributes to 
            in, out as well as bidirectional connections.
            
            It should also be noted that any connection types between a diad does not contribite to their
            social resemblance. In other terms their structual embeddedness in the network, in other words,
            their connection style with third nodes in the network is considered.
            
            It should be noted that the algorithm puts a very high importance on the connections with members.
        
        Args:
            egonet (:obj:`dict`): The ego net of the other agent.Expected keys are 'in','out','bi','ext'.
            
        Returns:
            (:obj:`float`): The social resemblance score.
            
        """
        egoA = {'bi': self.egonet['bi'],
                'in': self.egonet['in'].union(self.egonet['bi']),
                'out': self.egonet['out'].union(self.egonet['bi']),
                'ext': self.egonet['ext']
               }
        egoB = {'bi': egonet['bi'],
                'in': egonet['in'].union(egonet['bi']),
                'out': egonet['out'].union(egonet['bi']),
                'ext': egonet['ext']
               }
        score = 0
        for k in self.egonet.keys():
            setA = egoA[k]
            setB = egoB[k]
            c = self.community.compare(setA, setB, raw = False, exclusive = True)
            joint = c[0]
            dif_ab = c[1]
            dif_ba = c[2]
            score += joint * (1 - abs(dif_ab - dif_ba))
        return score
    
    
    def social_difference(self, egonet):
        """Agent compares her social connections to the social connections of another agent.
        She measures the social difference the other agent has. This measure can be considered
        as a proxy to measure the potential of extending social capital via the other agent.
        
        Note:
            The measure transforms the directed ego connections into undirected egocentric graph.
            It should be noted connections to non-community members are also considered.
            
        Args:
            egonet (:obj:`dict`): The ego net of the other agent.Expected keys are 'in','out','bi','ext'.
            
        Returns:
            (:obj:`float`): The social difference score.
            
        """
        setA = set()
        for k in self.egonet.keys(): setA = setA.union(self.egonet[k])
        setB = set()
        for k in egonet.keys(): setB = setB.union(egonet[k])
        dif_ba = self.community.compare(setA, setB, raw = False, exclusive = True)[2]
        return dif_ba
            
    
    def social_affiliation(self, egonet):
        """Agent considers her social interaction with the other agent and produces a social affiliation score.
        The behavioral preferences of the agent in the score generation is used.
        
        Note: In this current implementation social media interaction as of friendship stata are used. In the 
        behavioural preferences importance to reciprocity or unidirectional connection types are considered.
        
        $(ismutual x Wm + isfollowed x Wf + isfollowedby x Wb)$
        
        Args:
            
        Returns:
            
        """
        pass
    

In [204]:
MEMBER_PROFILES = load_member_profiles('./data/crm_profile.pickle')
aprofile = get_random_item(MEMBER_PROFILES)
aid = list(aprofile.keys())[0]
profile = aprofile[aid]
profile['id'] = aid
pp.pprint(profile)

{'entity': 'P',
'gender': 'M',
'id': '158179681',
'location': 'IT',
'tags': {'product', 'innovator', 'mechatronics', 'designer', 'engineer'},
'twitter': 'LorenzoDuroux'}


In [209]:
M = Maker(aid1,CE,SG)

In [210]:
get_attributes(M)

['id', 'state', 'type', 'modCogExp', 'egonet', 'community']

In [211]:
M.skills

set()

In [212]:
profile['skills'] = profile['tags']
M.load_profile(profile)
M.skills

{'designer', 'engineer', 'innovator', 'mechatronics', 'product'}

In [215]:
M.modCogExp.get_compatibility('3d', '3dmodel'), M.modCogExp.get_compatibility('3dmodel', '3d')

(0.2, 0.03571428571428571)

In [216]:
M.modCogExp.get_compatibility('3d', 'audio')

0.0

In [221]:
M.skill_compatibility(['3d', 'opendesign', 'solutions'])

0.45

In [222]:
M.skill_similarity(['3d', 'audio', 'opendesign', 'solutions'])

0.051461693548387095

In [223]:
M.skill_difference(['3d',  'opendesign', 'research', 'solutions'])

13.15154398758526

In [225]:
a = '3d'
b = '3dmodel'

In [227]:
M.modCogExp.get_centrality(a), M.modCogExp.get_centrality(b)

(28, 5)

In [228]:
M.modCogExp.get_diffusion(a), M.modCogExp.get_diffusion(b)

(28, 5)

In [229]:
M.modCogExp.get_distinctiveness(a), M.modCogExp.get_distinctiveness(b)

(1.0, 1.0)

In [230]:
M.modCogExp.get_compatibility(a, b), M.modCogExp.get_compatibility(b, a)

(0.2, 0.03571428571428571)

In [231]:
M.modCogExp.get_similarity(a,b,raw=False,exclusive=False), M.modCogExp.get_similarity(b, a, raw=False,exclusive=False)

(0.03125, 0.03125)

In [232]:
M.modCogExp.get_similarity(a,b,raw=False,exclusive=True), M.modCogExp.get_similarity(b,a,raw=False,exclusive=True)

(0.03571428571428571, 0.2)

In [233]:
M.modCogExp.get_difference(a,b,raw=False,exclusive=True), M.modCogExp.get_difference(b,a,raw=False,exclusive=True)

(0.9642857142857143, 0.8)

In [234]:
M.modCogExp.get_difference(a,b,raw=False,exclusive=False), M.modCogExp.get_difference(b,a,raw=False,exclusive=False)

(0.84375, 0.125)

In [235]:
M.modCogExp.get_resemblance(a, b)

0.02611607142857143

In [236]:
c = '4d'
newprojs = [[b, a], [c, a]]
M.modCogExp.update_relevance_models(newprojs)

True

In [237]:
print(M.modCogExp.get_centrality(a), M.modCogExp.get_centrality(b))
M.modCogExp.get_similarity(a,b,raw=False,exclusive=True), M.modCogExp.get_similarity(b,a,raw=False,exclusive=True)

30 6


(0.06666666666666667, 0.3333333333333333)

In [238]:
M.modCogExp.get_compatibility(c, a), M.modCogExp.get_compatibility(c, b)

(0.03333333333333333, 0.0)

In [241]:
M.modCogExp.get_centrality(a), CE.get_centrality(b), CE.get_centrality(c)

(30, 6, 1)

In [242]:
CE.get_distinctiveness(a), M.modCogExp.get_distinctiveness(b), M.modCogExp.get_distinctiveness(c)

(1.0344827586206897, 1.0, 1.0)

In [243]:
newprofile = [a,b,c]
M.modCogExp.update_diffusion_levels(newprofile)

True

In [244]:
M.modCogExp.get_distinctiveness(a), M.modCogExp.get_distinctiveness(b), M.modCogExp.get_distinctiveness(c)

(1.0, 0.8571428571428571, 0.5)

In [245]:
help(M.skill_compatibility)

Help on method skill_compatibility in module __main__:

skill_compatibility(skills) method of __main__.Maker instance
    Comparing agent's own declared skills to the given an external profile, agent
        measure the compatibility of the external profile to her profile considering
        the knowledge the agent has about the projects in the community.
    
    Args:
        skills (:obj:`list` of :obj:`str`): The skill set of the other agent.
        
    Returns:
        (:obj:`float`): The compatibility score.



In [248]:
M.skill_compatibility(['3d', 'design', 'innovation'])

2.0

In [249]:
help(M.social_resemblance)

Help on method social_resemblance in module __main__:

social_resemblance(egonet) method of __main__.Maker instance
    Agent compares her social connections to the social connections of another agent. She measures her 
    structural equivalance to the other agent, both in terms of similarities as well as differences.
    
    Note:
        The algorithm below puts a positive bias at being on the same k-cliques for k > 2 as contributes to 
        in, out as well as bidirectional connections.
        
        It should also be noted that any connection types between a diad does not contribite to their
        social resemblance. In other terms their structual embeddedness in the network, in other words,
        their connection style with third nodes in the network is considered.
        
        It should be noted that the algorithm puts a very high importance on the connections with members.
    
    Args:
        egonet (:obj:`dict`): The ego net of the other agent.Expected keys ar

In [251]:
M.social_resemblance(egoA)

4.0

In [252]:
M.social_resemblance(egoB)

0.0690928270576597